# <font color='black'><center><center></font> 
![title](img/All.png) 
# <font color='white'><center><center></font> 
# <font><center>Artificial Intelligence<center></font>
## <font ><center>Hospital Job Scheduling<center></font>
## <center>Teacher: Dr. Fadaei<center>
### <center>Student Name: Mohammadali Shakerdargah<center>
### <center>Student Number: 810098002<center>


### <font>Purpose:</font>
    In this project we are going to use Arversarial-Game algorithems and try to make an agent that would win most of the times in a gane which is played similar to blackjack  
    
### <font>What  we did in this Hands-On:</font>
    In this Hands-On we used "Arversarial-Game algorithems" made an agent that would win most of the times in a gane which is played similar to blackjack  
    
#### <font>Part-1:</font>
    At first we will explain why why minimax algorithm does NOT work when we don't know the next cards and we will demonstrate a solution to this problem
    
#### <font>Part-2:</font>
    This part will be explained in wo sub-parts
    sub-part 2-1: minimax algorithm
    sub-part 2-2: pruning algorithm

#### <font>Part-3:</font>
    In this part we are going to talk about the way we developed our code in part 2
        

## Part 1: Replacement for minimax 

In this part we will discuss why minimax won't work when we don't know the next cards of our deck:
When we use minimax algorithm, our agent is just in the "THINKING" state, it thinks if I do action A what would happen and goes to beneath levels.

![title](img/AgentThinking.png) 

Solution: Using Expectimax instead of Minimax.

The Expectimax search algorithm is a game theory algorithm used to maximize the expected utility. It is a variation of the Minimax algorithm. While Minimax assumes that the adversary(the minimizer) plays optimally, the Expectimax doesn’t. This is useful for modelling environments where adversary agents are not optimal, or their actions are based on chance.

So we would assume that it is completely stochastic


In [17]:
import random

In [56]:
!jt
!jt -t onedork -T -N -kl

'jt' is not recognized as an internal or external command,
operable program or batch file.
'jt' is not recognized as an internal or external command,
operable program or batch file.


## Part 2-1: Minimax without pruning 

In this part we used minimax algorithm to predict the best option for our agent to take action.
In this implementation I added 4 functions and 1 class.

- HypGame_Player class: makes a Hypothetic player state and inherets its features from parent

- utility function: based on a heuristic gives us the rate of an action

- minimax_decision: gives the best move for our action 

- max_value: based on 4 actions, continues building the tree and takes the most benefitial action for the player

- min_value: based on 4 actions, continues building the tree and takes the most benefitial action for the opponent

In [18]:
class Player:
    def __init__(self, name, num_of_cards):
        """
        The base player class of the game
        Inputs
        -----------
        name = (str) player's name
        num_of_cards = (int) number of cards in the deck
        """
        self.name = name
        self.deck_count = num_of_cards
        self.target = self.deck_count * 2 - 1
        self.cards = []
        self.erases_remaining = self.deck_count // 5
        self.has_stopped = False

    def draw_card(self, card):
        """
        draws a card, and adds it to player cards
        Input
        -------------
        card: (int) the card to be added
        """
        self.cards.append(card)

    def print_info(self):
        """
        prints info of the player
        """
        print(f"{self.name}'s cards: ", end='')
        for c in self.cards:
            print(f'{c}, ', end='')
        print(f'sum: {sum(self.cards)}')
    
    def get_margin(self):
        """
        returns the margin left to target by the player
        Output
        ----------
        (int) margin to target
        """
        return self.target - sum(self.cards)
    
    def cpu_play(self, seen_cards, deck, enemies_cards):
        """
        The function for cpu to play the game
        Inputs
        ----------
        seen_cards:     (list of ints) the cards that have been seen until now
        deck:           (list of ints) the remaining playing deck of the game
        enemies_cards:  (list of ints) the cards that the enemy currently has.
        Output
        ----------
        (str) a command given to the game
        
        """
        if (len(deck) > 0):
            next_card_in_deck = deck[0]
        else:
            next_card_in_deck = 0
        if (len(deck) > 1):
            next_enemy_card_in_deck = deck[1]
        else:
            next_enemy_card_in_deck = 0
        amount_to_target = self.target - sum(self.cards)
        amount_with_next_card = self.target - (sum(self.cards) + next_card_in_deck)
        enemies_amount_to_target = self.target - sum(enemies_cards)
        enemies_amount_with_next_card = self.target - (sum(enemies_cards) + next_enemy_card_in_deck)
        _stop_condition = amount_to_target < next_card_in_deck and self.erases_remaining <= 0
        _draw_condition_1 = next_card_in_deck != 0
        _draw_condition_2 = amount_with_next_card >= 0
        _erase_condition = self.erases_remaining > 0 
        _erase_self_condition = amount_to_target < 0 # sum(cards) > target
        _erase_opponent_condition_or = enemies_amount_to_target < (self.target // 7)
        _erase_opponent_condition_or_2 = enemies_amount_with_next_card < (self.target // 7) 
        _erase_opponent_condition_or_3 = enemies_amount_with_next_card <= amount_with_next_card
        _erase_opponent_condition_or_4 = enemies_amount_to_target <= amount_to_target
        _erase_opponent_condition = _erase_opponent_condition_or or _erase_opponent_condition_or_2 or _erase_opponent_condition_or_3
        _erase_opponent_condition = _erase_opponent_condition or _erase_opponent_condition_or_4 
        if (_stop_condition):
            return 'stop'
        elif (_draw_condition_1 and _draw_condition_2):
            return 'draw'
        elif(_erase_self_condition and _erase_condition):
            return 'erase_self'
        elif(_erase_opponent_condition and _erase_condition):
            return 'erase_opponent'
        else:
            return 'stop'
    
    def erase(self, target):
        """
        erases the last card of the target
        Input
        ---------
        target: (Player obj) the player whos last card is about to be erased
        """
        if (len(target.cards) == 0):
            print(f'{target.name} has no more eraseble cards!')
            return
        if (self.erases_remaining > 0):
            self.erases_remaining -= 1
            card = target.cards.pop(-1)
            print(f'{self.name} erased {card} from {target.name}\'s deck!')
            return
        print(f'{self.name} has no more erases remaining!')

    def get_player_cards(self):
        return self.cards

    def get_erases_remained(self):
        return self.erases_remaining
    
    def get_stoppedStatus(self):
        return self.has_stopped

In [19]:
class HypGame_Player:
    def __init__(self, name, parent):
        self.name = name
        self.target = 41
        self.cards = parent.get_player_cards()
        self.erases_remaining = parent.get_erases_remained()
        self.has_stopped = parent.get_erases_remained()
        
    def draw_card(self, card):
        self.cards.append(card)
    
    def get_margin(self):
        return self.target - sum(self.cards)
    
    def erase_self(self, target):
        if (len(target.cards) == 0):
            print(f'{target.name} has no more eraseble cards!')
            return
        if (self.erases_remaining > 0):
            self.erases_remaining -= 1
            card = target.cards.pop(-1)
            print(f'{self.name} erased {card} from {target.name}\'s deck!')
            return
        print(f'{self.name} has no more erases remaining!')

    def get_player_cards(self):
        return self.cards

    def get_erases_remained(self):
        return self.erases_remaining
    
    def erase_opponent(self, target):
        target_cards = target.get_player_cards()
        if (len(target_cards) == 0):
            print(f'{target.name} has no more eraseble cards!')
            return
        if (self.erases_remaining > 0):
            self.erases_remaining -= 1
            card = target_cards.pop(-1)
            target.set_player_cards(target_cards)
            print(f'{self.name} erased {card} from {target.name}\'s deck!')
            return
        print(f'{self.name} has no more erases remaining!')
    
    def set_stoppedStatus(self, _stopStatus):
        self.has_stopped = _stopStatus
    
    def get_stoppedStatus(self):
        return self.has_stopped
    
    def set_player_cards(self, _cards):
        self.cards = _cards

This Utility will give us the rate of that action, which is the difference of players margin

In [20]:
def utility(_player, _opponent):
    rate = _player.get_margin() - _opponent.get_margin()
    return rate

In [21]:
def minimax_decision(_player, _opponent, _deck, _depth):
    player = HypGame_Player('player', _player) 
    opponent = HypGame_Player('opponent', _opponent)
    deck = _deck
    depth = _depth
    maxRate, maxActions = max_value(player, opponent, deck, depth, 's') 
    return maxActions #'d','s','es','eo'

In [22]:
def max_value(_player, _opponent, _deck, _depth, _move):
    if(_depth <=0 or len(_deck)<=0): #reached max-depth or cards finished
        return utility(_player, _opponent), _move
    
    actions = ['d', 's', 'es','eo']
    rateOfActions = []
    player = HypGame_Player('player', _player) #Current state
    opponent = HypGame_Player('opponent', _opponent) #Current state
    for action in actions:
        if action == 'd':
            if(player.get_stoppedStatus() == False and len(_deck)>0):
                player.draw_card(_deck.pop(0))
            rateOfAction, action = min_value(player, opponent, _deck, _depth-1, 'd')
            rateOfActions.append(rateOfAction)
        if action == 's':
            if(player.get_stoppedStatus() == False):
                player.set_stoppedStatus(True)
            rateOfAction, action = min_value(player, opponent, _deck, _depth-1, 's')
            rateOfActions.append(rateOfAction)
        if action == 'es':
            if(player.get_stoppedStatus() == False and player.get_erases_remained()>0):
                player.erase_self(player)
            rateOfAction, action = min_value(player, opponent, _deck, _depth-1, 'es')
            rateOfActions.append(rateOfAction)
        if action == 'eo':
            if(player.get_stoppedStatus() == False and player.get_erases_remained()>0):
                player.erase_opponent(opponent)
            rateOfAction, action = min_value(player, opponent, _deck, _depth-1, 'eo')
            rateOfActions.append(rateOfAction)
    maxRate = max(rateOfActions)
    maxRateIndex = rateOfActions.index(max(rateOfActions))
    return maxRate, actions[maxRateIndex]

In [23]:
def min_value(_player, _opponent, _deck, _depth, _move):
    if(_depth <=0 or len(_deck)<=0): #reached max-depth or cards finished
        return utility(_player, _opponent), _move
    
    actions = ['d', 's', 'es','eo']
    rateOfActions = []
    player = HypGame_Player('player', _player) #Current state
    opponent = HypGame_Player('opponent', _opponent) #Current state
    for action in actions:
        if action == 'd':
            if(opponent.get_stoppedStatus() == False and len(_deck)>0):
                opponent.draw_card(_deck.pop(0))
            rateOfAction, action = max_value(player, opponent, _deck, _depth-1, 'd')
            rateOfActions.append(rateOfAction)
        if action == 's':
            if(opponent.get_stoppedStatus() == False):
                opponent.set_stoppedStatus(True)
            rateOfAction, action = max_value(player, opponent, _deck, _depth-1, 's')
            rateOfActions.append(rateOfAction)
        if action == 'es':
            if(opponent.get_stoppedStatus() == False and opponent.get_erases_remained()>0):
                opponent.erase_self(opponent)
            rateOfAction, action = max_value(player, opponent, _deck, _depth-1, 'es')
            rateOfActions.append(rateOfAction)
        if action == 'eo':
            if(opponent.get_stoppedStatus() == False and opponent.get_erases_remained()>0):
                opponent.erase_opponent(player)
            rateOfAction, action = max_value(player, opponent, _deck, _depth-1, 'eo')
            rateOfActions.append(rateOfAction)
    minRate = min(rateOfActions)
    minRateIndex = rateOfActions.index(min(rateOfActions))
    return minRate, actions[minRateIndex]

In [24]:
class Blacksin:
    def __init__(self, deck_count=21):
        """
        The main game class
        Inputs
        -----------
        deck_count = (int) number of cards in the deck
        """
        
        self.deck_count = deck_count
        self.target = self.deck_count * 2 - 1
        self.player = Player('player', deck_count)
        self.opponent = Player('opponent', deck_count)
        self.deck = self.shuffle_cards()
        self.seen_cards = []
    
    def shuffle_cards(self):
        """ 
        shuffles cards for deck creation
        """
        return list(random.sample(range(1, self.deck_count + 1), self.deck_count))

    def draw_card(self):
        """ 
        draws a card from deck, if non is remaining, ends the game.
        """
        if (len(self.deck) > 0):
            card = self.deck.pop(0)
            self.seen_cards.append(card)
            return card
        print('The deck is empty! ending game...')
        self.opponent.has_stopped = True
        self.player.has_stopped = True
        return -1

    def handout_cards(self):
        """ 
        hands out cards to players
        """ 
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
    
    def handle_input(self, _input, player):
        
        """ 
        handles input
        Input
        ------------
        _input: (str) input given by the player
        player: (Player obj)the player that is giving the input
        """
        
        if (player is self.player):
            opponent = self.opponent
        else:
            opponent = self.player
        if (_input == 'stop' or _input == 's'):
            player.has_stopped = True
            print(f'{player.name} has stopped')
        elif (_input == 'draw' or _input == 'd'):
            card = self.draw_card()
            if (card == -1): return True
            player.draw_card(card)
            print(f'{player.name} drawed a card: {card}')
        elif ((_input == 'erase_self' or _input == 'es')):
            player.erase(player)
        elif ((_input == 'erase_opponent' or _input == 'eo')):
            player.erase(opponent)
        else:
            print('ERROR: unknown command')
            return False
        return True

    def get_player_input(self):
        depth = 7
#         depth = 8
#         depth = 9
#         depth = 10
#         depth = 11
#         depth = 12
        your_input = minimax_decision(self.player, self.opponent, self.deck, depth)
        self.handle_input(your_input, self.player)
            
    def opponent_play(self):
        """
        function for opponent to play it's turn
        """
        try:
            opponent_input = self.opponent.cpu_play(self.seen_cards, self.deck, self.player.cards)
        except:
            opponent_input = 'stop'
        self.handle_input(opponent_input, self.opponent)

    def check_for_winners(self):
        """
        checks for winners.
        Output
        -----------
        (int) returns 1 if player wins, 0 if draw and -1 if opponent wins
        """
        self.opponent.print_info()
        self.player.print_info()
        player_margin = self.player.get_margin()
        opponent_margin = self.opponent.get_margin()
        player_win_condition_1 = opponent_margin < 0 and player_margin >= 0
        player_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin < opponent_margin
        draw_condition_1 = opponent_margin < 0 and player_margin < 0
        draw_condition_2 = opponent_margin >= 0 and player_margin >= 0 and player_margin == opponent_margin
        opponent_win_condition_1 = player_margin < 0 and opponent_margin >= 0
        opponent_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin > opponent_margin
        if (player_win_condition_1 or player_win_condition_2):
            print(f'the winner is the {self.player.name}!')
            return 1
        elif(draw_condition_1 or draw_condition_2):
            print('the game ends in a draw!')
            return 0
        elif(opponent_win_condition_1 or opponent_win_condition_2):
            print(f'the winner is the {self.opponent.name}!')
            return -1
        else:
            print('an error has accurred! exiting...')
            exit()

    def print_deck(self):
        """
        prints the current deck of the game
        """
        print('full deck: [top] ', end='')
        for i in self.deck:
            print(i, end=' ')
        print('[bottom]')

    def run(self):
        """
        main function to run the game with
        """
        print('\nstarting game... shuffling... handing out cards...')
        print(f'remember, you are aiming for nearest to: {self.target}')
        self.print_deck()
        self.handout_cards()
        turn = 0 
        while(not self.player.has_stopped or not self.opponent.has_stopped):
            if (turn == 0):
                if (not self.player.has_stopped):
                    self.opponent.print_info()
                    self.player.print_info()
                    self.get_player_input()
                    print()
            else:
                if (not self.opponent.has_stopped):
                    print('opponent playing...')
                    self.opponent_play()
                    print()
            turn = 1 - turn
        print('\nand the winner is...')
        return self.check_for_winners()

In [25]:
# Example of using the game class
cntPlayer = 0
cntOpponent = 0
for i in range(500):
    game = Blacksin(deck_count=21)
    result = game.run()
    if(result == 1):
        cntPlayer = cntPlayer+1
    elif(result == -1):
        cntOpponent = cntOpponent+1
    print(f"-------------------\n {i}")
print("Winning over opponent: ",(cntPlayer/cntOpponent),"times")


starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 16 12 7 18 6 3 4 5 20 17 10 8 9 1 19 15 14 11 13 21 2 [bottom]
opponent's cards: 12, 18, sum: 30
player's cards: 16, 7, sum: 23
player drawed a card: 6

opponent playing...
opponent drawed a card: 3

opponent's cards: 12, 18, 3, sum: 33
player's cards: 16, 7, 6, sum: 29
player drawed a card: 4

opponent playing...
opponent drawed a card: 5

opponent's cards: 12, 18, 3, 5, sum: 38
player's cards: 16, 7, 6, 4, sum: 33
player drawed a card: 20

opponent playing...
opponent erased 20 from player's deck!

opponent's cards: 12, 18, 3, 5, sum: 38
player's cards: 16, 7, 6, 4, sum: 33
player drawed a card: 17

opponent playing...
opponent erased 17 from player's deck!

opponent's cards: 12, 18, 3, 5, sum: 38
player's cards: 16, 7, 6, 4, sum: 33
player drawed a card: 10

opponent playing...
opponent erased 10 from player's deck!

opponent's cards: 12, 18, 3, 5, sum: 38
player's cards:

player drawed a card: 20

opponent playing...
opponent erased 20 from player's deck!

opponent's cards: 9, 12, 1, 4, 6, sum: 32
player's cards: 5, 19, 8, 2, 18, sum: 52
player drawed a card: 11

opponent playing...
opponent erased 11 from player's deck!

opponent's cards: 9, 12, 1, 4, 6, sum: 32
player's cards: 5, 19, 8, 2, 18, sum: 52
The deck is empty! ending game...


and the winner is...
opponent's cards: 9, 12, 1, 4, 6, 14, 15, 3, 16, 7, 13, 17, sum: 117
player's cards: 5, 19, 8, 2, 18, sum: 52
the game ends in a draw!
-------------------
 4

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 11 4 2 3 21 14 17 1 6 8 5 15 19 20 16 18 9 13 12 10 7 [bottom]
opponent's cards: 4, 3, sum: 7
player's cards: 11, 2, sum: 13
player drawed a card: 21

opponent playing...
opponent drawed a card: 14

opponent's cards: 4, 3, 14, sum: 21
player's cards: 11, 2, 21, sum: 34
player drawed a card: 17

opponent playing...
opponent drawed a 

player drawed a card: 9

opponent's cards: 16, 17, 6, sum: 39
player's cards: 1, 4, 3, 2, 11, 19, 10, 12, 8, 13, 18, 14, 9, sum: 124
player drawed a card: 15

opponent's cards: 16, 17, 6, sum: 39
player's cards: 1, 4, 3, 2, 11, 19, 10, 12, 8, 13, 18, 14, 9, 15, sum: 139
player drawed a card: 7

opponent's cards: 16, 17, 6, sum: 39
player's cards: 1, 4, 3, 2, 11, 19, 10, 12, 8, 13, 18, 14, 9, 15, 7, sum: 146
player drawed a card: 21

opponent's cards: 16, 17, 6, sum: 39
player's cards: 1, 4, 3, 2, 11, 19, 10, 12, 8, 13, 18, 14, 9, 15, 7, 21, sum: 167
player drawed a card: 20

opponent's cards: 16, 17, 6, sum: 39
player's cards: 1, 4, 3, 2, 11, 19, 10, 12, 8, 13, 18, 14, 9, 15, 7, 21, 20, sum: 187
player drawed a card: 5

opponent's cards: 16, 17, 6, sum: 39
player's cards: 1, 4, 3, 2, 11, 19, 10, 12, 8, 13, 18, 14, 9, 15, 7, 21, 20, 5, sum: 192
player has stopped


and the winner is...
opponent's cards: 16, 17, 6, sum: 39
player's cards: 1, 4, 3, 2, 11, 19, 10, 12, 8, 13, 18, 14, 9, 15,

player drawed a card: 10

opponent's cards: 8, 17, 15, sum: 40
player's cards: 5, 2, 1, 18, 13, 4, 11, 9, 12, 21, 7, 16, 19, 3, 6, 10, sum: 157
player drawed a card: 14

opponent's cards: 8, 17, 15, sum: 40
player's cards: 5, 2, 1, 18, 13, 4, 11, 9, 12, 21, 7, 16, 19, 3, 6, 10, 14, sum: 171
player drawed a card: 20

opponent's cards: 8, 17, 15, sum: 40
player's cards: 5, 2, 1, 18, 13, 4, 11, 9, 12, 21, 7, 16, 19, 3, 6, 10, 14, 20, sum: 191
player has stopped


and the winner is...
opponent's cards: 8, 17, 15, sum: 40
player's cards: 5, 2, 1, 18, 13, 4, 11, 9, 12, 21, 7, 16, 19, 3, 6, 10, 14, 20, sum: 191
the winner is the opponent!
-------------------
 13

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 18 20 16 9 14 15 11 17 4 5 7 1 13 12 19 2 8 21 10 6 3 [bottom]
opponent's cards: 20, 9, sum: 29
player's cards: 18, 16, sum: 34
player drawed a card: 14

opponent playing...
opponent erased 14 from player's deck!

opponent'

player drawed a card: 10

opponent playing...
opponent erased 10 from player's deck!

opponent's cards: 7, 15, 12, sum: 34
player's cards: 13, 4, 20, sum: 37
player drawed a card: 14

opponent playing...
opponent drawed a card: 2

opponent's cards: 7, 15, 12, 2, sum: 36
player's cards: 13, 4, 20, 14, sum: 51
player drawed a card: 21

opponent playing...
opponent erased 21 from player's deck!

opponent's cards: 7, 15, 12, 2, sum: 36
player's cards: 13, 4, 20, 14, sum: 51
player drawed a card: 19

opponent playing...
opponent erased 19 from player's deck!

opponent's cards: 7, 15, 12, 2, sum: 36
player's cards: 13, 4, 20, 14, sum: 51
The deck is empty! ending game...


and the winner is...
opponent's cards: 7, 15, 12, 2, 17, 18, 5, 11, 6, 1, 8, 3, 16, sum: 121
player's cards: 13, 4, 20, 14, sum: 51
the game ends in a draw!
-------------------
 19

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 2 13 5 7 8 19 10 11 18 12 20 9

...

player drawed a card: 14

opponent playing...
opponent erased 14 from player's deck!

opponent's cards: 16, 1, 19, sum: 36
player's cards: 7, 21, 20, sum: 48
player drawed a card: 15

opponent playing...
opponent erased 15 from player's deck!

opponent's cards: 16, 1, 19, sum: 36
player's cards: 7, 21, 20, sum: 48
player drawed a card: 10

opponent playing...
opponent erased 10 from player's deck!

opponent's cards: 16, 1, 19, sum: 36
player's cards: 7, 21, 20, sum: 48
player drawed a card: 18

opponent playing...
opponent drawed a card: 4

opponent's cards: 16, 1, 19, 4, sum: 40
player's cards: 7, 21, 20, 18, sum: 66
player drawed a card: 8

opponent playing...
opponent erased 8 from player's deck!

opponent's cards: 16, 1, 19, 4, sum: 40
player's cards: 7, 21, 20, 18, sum: 66
The deck is empty! ending game...


and the winner is...
opponent's cards: 16, 1, 19, 4, 17, 2, 6, 13, 5, 11, 12, 9, 3, sum: 118
player's cards: 7, 21, 20, 18, sum: 66
the game ends in a draw!
------------------

player drawed a card: 9

opponent playing...
opponent drawed a card: 6

opponent's cards: 21, 8, 6, sum: 35
player's cards: 4, 2, 9, sum: 15
player drawed a card: 12

opponent playing...
opponent erased 12 from player's deck!

opponent's cards: 21, 8, 6, sum: 35
player's cards: 4, 2, 9, sum: 15
player drawed a card: 19

opponent playing...
opponent erased 19 from player's deck!

opponent's cards: 21, 8, 6, sum: 35
player's cards: 4, 2, 9, sum: 15
player drawed a card: 13

opponent playing...
opponent has stopped

opponent's cards: 21, 8, 6, sum: 35
player's cards: 4, 2, 9, 13, sum: 28
player drawed a card: 14

opponent's cards: 21, 8, 6, sum: 35
player's cards: 4, 2, 9, 13, 14, sum: 42
player drawed a card: 5

opponent's cards: 21, 8, 6, sum: 35
player's cards: 4, 2, 9, 13, 14, 5, sum: 47
player drawed a card: 18

opponent's cards: 21, 8, 6, sum: 35
player's cards: 4, 2, 9, 13, 14, 5, 18, sum: 65
player drawed a card: 11

opponent's cards: 21, 8, 6, sum: 35
player's cards: 4, 2, 9, 13,

player drawed a card: 1

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, sum: 40
player drawed a card: 3

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, 3, sum: 43
player drawed a card: 13

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, 3, 13, sum: 56
player drawed a card: 18

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, 3, 13, 18, sum: 74
player drawed a card: 7

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, 3, 13, 18, 7, sum: 81
player drawed a card: 6

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, 3, 13, 18, 7, 6, sum: 87
player drawed a card: 4

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, 3, 13, 18, 7, 6, 4, sum: 91
player drawed a card: 5

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, 3, 13, 18, 7, 6, 4, 5, sum: 96
player drawed a card: 11

opponent's cards: 21, 19, sum: 40
player's cards: 14, 2, 15, 8, 1, 3, 13, 18,

player drawed a card: 19

opponent playing...
opponent drawed a card: 8

opponent's cards: 11, 1, 10, 8, sum: 30
player's cards: 15, 12, 4, 19, sum: 50
player drawed a card: 16

opponent playing...
opponent erased 16 from player's deck!

opponent's cards: 11, 1, 10, 8, sum: 30
player's cards: 15, 12, 4, 19, sum: 50
player drawed a card: 13

opponent playing...
opponent drawed a card: 3

opponent's cards: 11, 1, 10, 8, 3, sum: 33
player's cards: 15, 12, 4, 19, 13, sum: 63
player drawed a card: 14

opponent playing...
opponent erased 14 from player's deck!

opponent's cards: 11, 1, 10, 8, 3, sum: 33
player's cards: 15, 12, 4, 19, 13, sum: 63
player drawed a card: 20

opponent playing...
opponent drawed a card: 6

opponent's cards: 11, 1, 10, 8, 3, 6, sum: 39
player's cards: 15, 12, 4, 19, 13, 20, sum: 83
player drawed a card: 2

opponent playing...
opponent erased 2 from player's deck!

opponent's cards: 11, 1, 10, 8, 3, 6, sum: 39
player's cards: 15, 12, 4, 19, 13, 20, sum: 83
player dr

player drawed a card: 11

opponent's cards: 16, 4, 14, sum: 34
player's cards: 1, 15, 2, 8, 20, 10, 5, 9, 6, 12, 21, 13, 18, 3, 19, 11, sum: 173
player drawed a card: 7

opponent's cards: 16, 4, 14, sum: 34
player's cards: 1, 15, 2, 8, 20, 10, 5, 9, 6, 12, 21, 13, 18, 3, 19, 11, 7, sum: 180
player drawed a card: 17

opponent's cards: 16, 4, 14, sum: 34
player's cards: 1, 15, 2, 8, 20, 10, 5, 9, 6, 12, 21, 13, 18, 3, 19, 11, 7, 17, sum: 197
player has stopped


and the winner is...
opponent's cards: 16, 4, 14, sum: 34
player's cards: 1, 15, 2, 8, 20, 10, 5, 9, 6, 12, 21, 13, 18, 3, 19, 11, 7, 17, sum: 197
the winner is the opponent!
-------------------
 495

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 2 7 13 16 6 5 4 1 14 12 19 18 10 15 11 9 17 21 3 20 8 [bottom]
opponent's cards: 7, 16, sum: 23
player's cards: 2, 13, sum: 15
player drawed a card: 6

opponent playing...
opponent drawed a card: 5

opponent's cards: 7, 16

With Level of 7 This shows that our agent wins almost 73% more than opponent.
We tried 8, 9 and 10 levels but those were really time consuming and even though cards would shuffle but we will see deeper results and we can choose a better action

## Part 2-2: Minimax with pruning - alpha-betha

In this part we used Alpha-Beta Pruning algorithm, 

The variables α and β are used to keep track of the current upper and lower bounds, as we do the depth-first search.

- α is the lower bound on MAX's outcome
- β is the upper bound on MIN's outcome

Suppose that v is the value of some node we're looking at. If the path to this node is still viable, we must have α ≤ v ≤ β

We pass alpha and beta down in our depth-first search, returning prematurely when

- v>β or v<α (no viable path through this node), or
- v=β or v=α (no better path through this node).

Resource: https://courses.engr.illinois.edu/cs440/fa2018/lectures/lect33.html

We still have 1 more class and 4 functions as bellow: 

- HypGame_Player class: makes a Hypothetic player state and inherets its features from parent

- utility function: based on a heuristic gives us the rate of an action

- minimax_decision: gives the best move for our action 

- max_value: based on 4 actions, continues building the tree and takes the most benefitial action for the player

- min_value: based on 4 actions, continues building the tree and takes the most benefitial action for the opponent

In [40]:
def minimax_decision_pruning(_player, _opponent, _deck, _depth):
    player = HypGame_Player('player', _player) 
    opponent = HypGame_Player('opponent', _opponent)
    deck = _deck
    depth = _depth
    maxRate, maxActions = max_value_pruning(player, opponent, deck, depth, 's', -9999, 9999) 
    return maxActions #'d','s','es','eo'

In [45]:
def max_value_pruning(_player, _opponent, _deck, _depth, _move, alpha, betta):
    if(_depth <=0 or len(_deck)<=0): #reached max-depth or cards finished
        return utility(_player, _opponent), _move
    
    actions = ['d', 's', 'es','eo']
    rateOfActions = []
    player = HypGame_Player('player', _player) #Current state
    opponent = HypGame_Player('opponent', _opponent) #Current state
    rate = -9999
    
    for action in actions:
        if action == 'd':
            if(player.get_stoppedStatus() == False and len(_deck)>0):
                player.draw_card(_deck.pop(0))
            rateOfAction, action = min_value_pruning(player, opponent, _deck, _depth-1, 'd', alpha, betta)
            rate = max(rate, rateOfAction)
            if(rate >= betta):
                return rate, 'd'
            alpha = max(alpha, rate)
            rateOfActions.append(rateOfAction)
            
        if action == 's':
            if(player.get_stoppedStatus() == False):
                player.set_stoppedStatus(True)
            rateOfAction, action = min_value_pruning(player, opponent, _deck, _depth-1, 's', alpha, betta)
            rate = max(rate, rateOfAction)
            if(rate >= betta):
                return rate, 's'
            alpha = max(alpha, rate)
            rateOfActions.append(rateOfAction)
        
        if action == 'eo':
            if(player.get_stoppedStatus() == False and player.get_erases_remained()>0):
                player.erase_opponent(opponent)
            rateOfAction, action = min_value_pruning(player, opponent, _deck, _depth-1, 'eo', alpha, betta)
            rate = max(rate, rateOfAction)
            if(rate >= betta):
                return rate, 'eo'
            alpha = max(alpha, rate)
            rateOfActions.append(rateOfAction)
            
        if action == 'es':
            if(player.get_stoppedStatus() == False and player.get_erases_remained()>0):
                player.erase_self(player)
            rateOfAction, action = min_value_pruning(player, opponent, _deck, _depth-1, 'es', alpha, betta)
            rate = max(rate, rateOfAction)
            if(rate >= betta):
                return rate, 'es'
            alpha = max(alpha, rate)
            rateOfActions.append(rateOfAction)
            
    maxRate = max(rateOfActions)
    maxRateIndex = rateOfActions.index(max(rateOfActions))
    return maxRate, actions[maxRateIndex]

In [46]:
def min_value_pruning(_player, _opponent, _deck, _depth, _move, alpha, betta):
    if(_depth <=0 or len(_deck)<=0): #reached max-depth or cards finished
        return utility(_player, _opponent), _move
    
    actions = ['d', 's', 'es','eo']
    rateOfActions = []
    player = HypGame_Player('player', _player) #Current state
    opponent = HypGame_Player('opponent', _opponent) #Current state
    rate = 9999
    
    for action in actions:
        if action == 'd':
            if(opponent.get_stoppedStatus() == False and len(_deck)>0):
                opponent.draw_card(_deck.pop(0))
            rateOfAction, action = max_value_pruning(player, opponent, _deck, _depth-1, 'd', alpha, betta)
            rate = min(rate, rateOfAction)
            if(rate <= alpha):
                return rate, 'd'
            betta = min(betta, rate)
            
            rateOfActions.append(rateOfAction)
        if action == 's':
            if(opponent.get_stoppedStatus() == False):
                opponent.set_stoppedStatus(True)
            rateOfAction, action = max_value_pruning(player, opponent, _deck, _depth-1, 's', alpha, betta)
            rate = min(rate, rateOfAction)
            if(rate <= alpha):
                return rate, 's'
            betta = min(betta, rate)
            
            rateOfActions.append(rateOfAction)
            
        if action == 'eo':
            if(opponent.get_stoppedStatus() == False and opponent.get_erases_remained()>0):
                opponent.erase_opponent(player)
            rateOfAction, action = max_value_pruning(player, opponent, _deck, _depth-1, 'eo', alpha, betta)
            rate = min(rate, rateOfAction)
            if(rate <= alpha):
                return rate, 'eo'
            betta = min(betta, rate)
            
            rateOfActions.append(rateOfAction)
            
        if action == 'es':
            if(opponent.get_stoppedStatus() == False and opponent.get_erases_remained()>0):
                opponent.erase_self(opponent)
            rateOfAction, action = max_value_pruning(player, opponent, _deck, _depth-1, 'es', alpha, betta)
            rate = min(rate, rateOfAction)
            if(rate <= alpha):
                return rate, 'es'
            betta = min(betta, rate)
            
            rateOfActions.append(rateOfAction)
        
    minRate = min(rateOfActions)
    minRateIndex = rateOfActions.index(min(rateOfActions))
    return minRate, actions[minRateIndex]

In [48]:
class Blacksin_purring:
    def __init__(self, deck_count=21):
        """
        The main game class
        Inputs
        -----------
        deck_count = (int) number of cards in the deck
        """
        
        self.deck_count = deck_count
        self.target = self.deck_count * 2 - 1
        self.player = Player('player', deck_count)
        self.opponent = Player('opponent', deck_count)
        self.deck = self.shuffle_cards()
        self.seen_cards = []
    
    def shuffle_cards(self):
        """ 
        shuffles cards for deck creation
        """
        return list(random.sample(range(1, self.deck_count + 1), self.deck_count))

    def draw_card(self):
        """ 
        draws a card from deck, if non is remaining, ends the game.
        """
        if (len(self.deck) > 0):
            card = self.deck.pop(0)
            self.seen_cards.append(card)
            return card
        print('The deck is empty! ending game...')
        self.opponent.has_stopped = True
        self.player.has_stopped = True
        return -1

    def handout_cards(self):
        """ 
        hands out cards to players
        """ 
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
    
    def handle_input(self, _input, player):
        
        """ 
        handles input
        Input
        ------------
        _input: (str) input given by the player
        player: (Player obj)the player that is giving the input
        """
        
        if (player is self.player):
            opponent = self.opponent
        else:
            opponent = self.player
        if (_input == 'stop' or _input == 's'):
            player.has_stopped = True
            print(f'{player.name} has stopped')
        elif (_input == 'draw' or _input == 'd'):
            card = self.draw_card()
            if (card == -1): return True
            player.draw_card(card)
            print(f'{player.name} drawed a card: {card}')
        elif ((_input == 'erase_self' or _input == 'es')):
            player.erase(player)
        elif ((_input == 'erase_opponent' or _input == 'eo')):
            player.erase(opponent)
        else:
            print('ERROR: unknown command')
            return False
        return True

    def get_player_input(self):
        depth = 7
#         depth = 8
#         depth = 9
#         depth = 10
#         depth = 11
#         depth = 12
        your_input = minimax_decision_pruning(self.player, self.opponent, self.deck, depth)
        self.handle_input(your_input, self.player)
        
    def opponent_play(self):
        """
        function for opponent to play it's turn
        """
        try:
            opponent_input = self.opponent.cpu_play(self.seen_cards, self.deck, self.player.cards)
        except:
            opponent_input = 'stop'
        self.handle_input(opponent_input, self.opponent)

    def check_for_winners(self):
        """
        checks for winners.
        Output
        -----------
        (int) returns 1 if player wins, 0 if draw and -1 if opponent wins
        """
        self.opponent.print_info()
        self.player.print_info()
        player_margin = self.player.get_margin()
        opponent_margin = self.opponent.get_margin()
        player_win_condition_1 = opponent_margin < 0 and player_margin >= 0
        player_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin < opponent_margin
        draw_condition_1 = opponent_margin < 0 and player_margin < 0
        draw_condition_2 = opponent_margin >= 0 and player_margin >= 0 and player_margin == opponent_margin
        opponent_win_condition_1 = player_margin < 0 and opponent_margin >= 0
        opponent_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin > opponent_margin
        if (player_win_condition_1 or player_win_condition_2):
            print(f'the winner is the {self.player.name}!')
            return 1
        elif(draw_condition_1 or draw_condition_2):
            print('the game ends in a draw!')
            return 0
        elif(opponent_win_condition_1 or opponent_win_condition_2):
            print(f'the winner is the {self.opponent.name}!')
            return -1
        else:
            print('an error has accurred! exiting...')
            exit()

    def print_deck(self):
        """
        prints the current deck of the game
        """
        print('full deck: [top] ', end='')
        for i in self.deck:
            print(i, end=' ')
        print('[bottom]')

    def run(self):
        """
        main function to run the game with
        """
        print('\nstarting game... shuffling... handing out cards...')
        print(f'remember, you are aiming for nearest to: {self.target}')
        self.print_deck()
        self.handout_cards()
        turn = 0 
        while(not self.player.has_stopped or not self.opponent.has_stopped):
            if (turn == 0):
                if (not self.player.has_stopped):
                    self.opponent.print_info()
                    self.player.print_info()
                    self.get_player_input()
                    print()
            else:
                if (not self.opponent.has_stopped):
                    print('opponent playing...')
                    self.opponent_play()
                    print()
            turn = 1 - turn
        print('\nand the winner is...')
        return self.check_for_winners()

In [49]:
# Example of using the game class
cntPlayer = 0
cntOpponent = 0
for i in range(500):
    game = Blacksin_purring(deck_count=21)
    result = game.run()
    if(result == 1):
        cntPlayer = cntPlayer+1
    elif(result == -1):
        cntOpponent = cntOpponent+1
    print(f"-------------------\n {i}")
print("Winning over opponent: ",(cntPlayer/cntOpponent),"times")


starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 21 1 8 16 10 15 11 14 4 3 20 18 13 19 2 6 17 7 9 12 5 [bottom]
opponent's cards: 1, 16, sum: 17
player's cards: 21, 8, sum: 29
player drawed a card: 10

opponent playing...
opponent drawed a card: 15

opponent's cards: 1, 16, 15, sum: 32
player's cards: 21, 8, 10, sum: 39
player drawed a card: 11

opponent playing...
opponent erased 11 from player's deck!

opponent's cards: 1, 16, 15, sum: 32
player's cards: 21, 8, 10, sum: 39
player drawed a card: 14

opponent playing...
opponent drawed a card: 4

opponent's cards: 1, 16, 15, 4, sum: 36
player's cards: 21, 8, 10, 14, sum: 53
player drawed a card: 3

opponent playing...
opponent erased 3 from player's deck!

opponent's cards: 1, 16, 15, 4, sum: 36
player's cards: 21, 8, 10, 14, sum: 53
player drawed a card: 20

opponent playing...
opponent erased 20 from player's deck!

opponent's cards: 1, 16, 15, 4, sum: 36
player's cards:

remember, you are aiming for nearest to: 41
full deck: [top] 8 13 9 14 1 6 15 2 5 19 11 16 21 17 7 3 20 12 4 18 10 [bottom]
opponent's cards: 13, 14, sum: 27
player's cards: 8, 9, sum: 17
player drawed a card: 1

opponent playing...
opponent drawed a card: 6

opponent's cards: 13, 14, 6, sum: 33
player's cards: 8, 9, 1, sum: 18
player drawed a card: 15

opponent playing...
opponent drawed a card: 2

opponent's cards: 13, 14, 6, 2, sum: 35
player's cards: 8, 9, 1, 15, sum: 33
player drawed a card: 5

opponent playing...
opponent erased 5 from player's deck!

opponent's cards: 13, 14, 6, 2, sum: 35
player's cards: 8, 9, 1, 15, sum: 33
player drawed a card: 19

opponent playing...
opponent erased 19 from player's deck!

opponent's cards: 13, 14, 6, 2, sum: 35
player's cards: 8, 9, 1, 15, sum: 33
player drawed a card: 11

opponent playing...
opponent erased 11 from player's deck!

opponent's cards: 13, 14, 6, 2, sum: 35
player's cards: 8, 9, 1, 15, sum: 33
player drawed a card: 16

opponen

player drawed a card: 17

opponent playing...
opponent drawed a card: 8

opponent's cards: 16, 3, 8, sum: 27
player's cards: 2, 19, 17, sum: 38
player drawed a card: 11

opponent playing...
opponent erased 11 from player's deck!

opponent's cards: 16, 3, 8, sum: 27
player's cards: 2, 19, 17, sum: 38
player drawed a card: 18

opponent playing...
opponent erased 18 from player's deck!

opponent's cards: 16, 3, 8, sum: 27
player's cards: 2, 19, 17, sum: 38
player drawed a card: 15

opponent playing...
opponent drawed a card: 6

opponent's cards: 16, 3, 8, 6, sum: 33
player's cards: 2, 19, 17, 15, sum: 53
player drawed a card: 4

opponent playing...
opponent erased 4 from player's deck!

opponent's cards: 16, 3, 8, 6, sum: 33
player's cards: 2, 19, 17, 15, sum: 53
player drawed a card: 14

opponent playing...
opponent erased 14 from player's deck!

opponent's cards: 16, 3, 8, 6, sum: 33
player's cards: 2, 19, 17, 15, sum: 53
The deck is empty! ending game...


and the winner is...
opponent

full deck: [top] 14 17 19 8 13 1 15 4 21 5 16 3 2 9 10 11 12 20 7 6 18 [bottom]
opponent's cards: 17, 8, sum: 25
player's cards: 14, 19, sum: 33
player drawed a card: 13

opponent playing...
opponent drawed a card: 1

opponent's cards: 17, 8, 1, sum: 26
player's cards: 14, 19, 13, sum: 46
player drawed a card: 15

opponent playing...
opponent drawed a card: 4

opponent's cards: 17, 8, 1, 4, sum: 30
player's cards: 14, 19, 13, 15, sum: 61
player drawed a card: 21

opponent playing...
opponent drawed a card: 5

opponent's cards: 17, 8, 1, 4, 5, sum: 35
player's cards: 14, 19, 13, 15, 21, sum: 82
player drawed a card: 16

opponent playing...
opponent drawed a card: 3

opponent's cards: 17, 8, 1, 4, 5, 3, sum: 38
player's cards: 14, 19, 13, 15, 21, 16, sum: 98
player drawed a card: 2

opponent playing...
opponent erased 2 from player's deck!

opponent's cards: 17, 8, 1, 4, 5, 3, sum: 38
player's cards: 14, 19, 13, 15, 21, 16, sum: 98
player drawed a card: 9

opponent playing...
opponent er

full deck: [top] 8 3 13 17 18 6 21 4 9 7 11 16 15 5 20 19 1 12 2 14 10 [bottom]
opponent's cards: 3, 17, sum: 20
player's cards: 8, 13, sum: 21
player drawed a card: 18

opponent playing...
opponent drawed a card: 6

opponent's cards: 3, 17, 6, sum: 26
player's cards: 8, 13, 18, sum: 39
player drawed a card: 21

opponent playing...
opponent drawed a card: 4

opponent's cards: 3, 17, 6, 4, sum: 30
player's cards: 8, 13, 18, 21, sum: 60
player drawed a card: 9

opponent playing...
opponent drawed a card: 7

opponent's cards: 3, 17, 6, 4, 7, sum: 37
player's cards: 8, 13, 18, 21, 9, sum: 69
player drawed a card: 11

opponent playing...
opponent erased 11 from player's deck!

opponent's cards: 3, 17, 6, 4, 7, sum: 37
player's cards: 8, 13, 18, 21, 9, sum: 69
player drawed a card: 16

opponent playing...
opponent erased 16 from player's deck!

opponent's cards: 3, 17, 6, 4, 7, sum: 37
player's cards: 8, 13, 18, 21, 9, sum: 69
player drawed a card: 15

opponent playing...
opponent erased 15 

...

player's cards: 15, 3, 4, 19, 9, 16, sum: 66
player drawed a card: 5

opponent's cards: 8, 20, sum: 28
player's cards: 15, 3, 4, 19, 9, 16, 5, sum: 71
player drawed a card: 6

opponent's cards: 8, 20, sum: 28
player's cards: 15, 3, 4, 19, 9, 16, 5, 6, sum: 77
player drawed a card: 13

opponent's cards: 8, 20, sum: 28
player's cards: 15, 3, 4, 19, 9, 16, 5, 6, 13, sum: 90
player drawed a card: 11

opponent's cards: 8, 20, sum: 28
player's cards: 15, 3, 4, 19, 9, 16, 5, 6, 13, 11, sum: 101
player drawed a card: 1

opponent's cards: 8, 20, sum: 28
player's cards: 15, 3, 4, 19, 9, 16, 5, 6, 13, 11, 1, sum: 102
player drawed a card: 2

opponent's cards: 8, 20, sum: 28
player's cards: 15, 3, 4, 19, 9, 16, 5, 6, 13, 11, 1, 2, sum: 104
player drawed a card: 21

opponent's cards: 8, 20, sum: 28
player's cards: 15, 3, 4, 19, 9, 16, 5, 6, 13, 11, 1, 2, 21, sum: 125
player drawed a card: 17

opponent's cards: 8, 20, sum: 28
player's cards: 15, 3, 4, 19, 9, 16, 5, 6, 13, 11, 1, 2, 21, 17, sum: 142


full deck: [top] 1 7 6 20 18 3 10 16 2 12 17 15 8 21 4 19 5 14 11 9 13 [bottom]
opponent's cards: 7, 20, sum: 27
player's cards: 1, 6, sum: 7
player drawed a card: 18

opponent playing...
opponent drawed a card: 3

opponent's cards: 7, 20, 3, sum: 30
player's cards: 1, 6, 18, sum: 25
player drawed a card: 10

opponent playing...
opponent erased 10 from player's deck!

opponent's cards: 7, 20, 3, sum: 30
player's cards: 1, 6, 18, sum: 25
player drawed a card: 16

opponent playing...
opponent drawed a card: 2

opponent's cards: 7, 20, 3, 2, sum: 32
player's cards: 1, 6, 18, 16, sum: 41
player drawed a card: 12

opponent playing...
opponent erased 12 from player's deck!

opponent's cards: 7, 20, 3, 2, sum: 32
player's cards: 1, 6, 18, 16, sum: 41
player drawed a card: 17

opponent playing...
opponent erased 17 from player's deck!

opponent's cards: 7, 20, 3, 2, sum: 32
player's cards: 1, 6, 18, 16, sum: 41
player drawed a card: 15

opponent playing...
opponent drawed a card: 8

opponent's

player drawed a card: 20

opponent playing...
opponent erased 20 from player's deck!

opponent's cards: 3, 18, 5, 1, 8, sum: 35
player's cards: 19, 10, 21, 4, 15, sum: 69
player drawed a card: 12

opponent playing...
opponent drawed a card: 6

opponent's cards: 3, 18, 5, 1, 8, 6, sum: 41
player's cards: 19, 10, 21, 4, 15, 12, sum: 81
player drawed a card: 13

opponent playing...
opponent erased 13 from player's deck!

opponent's cards: 3, 18, 5, 1, 8, 6, sum: 41
player's cards: 19, 10, 21, 4, 15, 12, sum: 81
player drawed a card: 2

opponent playing...
opponent erased 2 from player's deck!

opponent's cards: 3, 18, 5, 1, 8, 6, sum: 41
player's cards: 19, 10, 21, 4, 15, 12, sum: 81
player drawed a card: 9

opponent playing...
opponent erased 9 from player's deck!

opponent's cards: 3, 18, 5, 1, 8, 6, sum: 41
player's cards: 19, 10, 21, 4, 15, 12, sum: 81
The deck is empty! ending game...


and the winner is...
opponent's cards: 3, 18, 5, 1, 8, 6, 14, 16, 17, 11, 7, sum: 106
player's car

player's cards: 17, 3, 7, 10, 2, 15, 14, 16, 4, 12, sum: 100
player drawed a card: 11

opponent's cards: 20, 19, sum: 39
player's cards: 17, 3, 7, 10, 2, 15, 14, 16, 4, 12, 11, sum: 111
player drawed a card: 6

opponent's cards: 20, 19, sum: 39
player's cards: 17, 3, 7, 10, 2, 15, 14, 16, 4, 12, 11, 6, sum: 117
player drawed a card: 9

opponent's cards: 20, 19, sum: 39
player's cards: 17, 3, 7, 10, 2, 15, 14, 16, 4, 12, 11, 6, 9, sum: 126
player drawed a card: 18

opponent's cards: 20, 19, sum: 39
player's cards: 17, 3, 7, 10, 2, 15, 14, 16, 4, 12, 11, 6, 9, 18, sum: 144
player drawed a card: 13

opponent's cards: 20, 19, sum: 39
player's cards: 17, 3, 7, 10, 2, 15, 14, 16, 4, 12, 11, 6, 9, 18, 13, sum: 157
player drawed a card: 21

opponent's cards: 20, 19, sum: 39
player's cards: 17, 3, 7, 10, 2, 15, 14, 16, 4, 12, 11, 6, 9, 18, 13, 21, sum: 178
player drawed a card: 8

opponent's cards: 20, 19, sum: 39
player's cards: 17, 3, 7, 10, 2, 15, 14, 16, 4, 12, 11, 6, 9, 18, 13, 21, 8, sum:

opponent's cards: 4, 5, 16, 7, sum: 32
player's cards: 18, 11, 6, 15, sum: 50
player drawed a card: 19

opponent playing...
opponent drawed a card: 2

opponent's cards: 4, 5, 16, 7, 2, sum: 34
player's cards: 18, 11, 6, 15, 19, sum: 69
player drawed a card: 12

opponent playing...
opponent erased 12 from player's deck!

opponent's cards: 4, 5, 16, 7, 2, sum: 34
player's cards: 18, 11, 6, 15, 19, sum: 69
player drawed a card: 8

opponent playing...
opponent erased 8 from player's deck!

opponent's cards: 4, 5, 16, 7, 2, sum: 34
player's cards: 18, 11, 6, 15, 19, sum: 69
player drawed a card: 21

opponent playing...
opponent drawed a card: 1

opponent's cards: 4, 5, 16, 7, 2, 1, sum: 35
player's cards: 18, 11, 6, 15, 19, 21, sum: 90
player drawed a card: 14

opponent playing...
opponent erased 14 from player's deck!

opponent's cards: 4, 5, 16, 7, 2, 1, sum: 35
player's cards: 18, 11, 6, 15, 19, 21, sum: 90
The deck is empty! ending game...


and the winner is...
opponent's cards: 4, 5, 

With Level of 7 This shows that our agent wins almost 1.43 times than opponent(almost 50% more!). We tried 8, 9 and 10 levels but those were really time consuming and even though cards would shuffle but we will see deeper results and we can choose a better action.
The difference here was that although our operation was faster but we gained a lower amount of wins.

## Part 3: Answering questions:


- I decided to to use 'd', 's', 'eo', 'es'

- Of course IT MATTERS, because th fist child of every node will go on and on till the terminal test, after that, the value of first node will be compared to other nodes and if it is too low and the level was trying to maximize, it will be not benificial and if it was too high and the level was minimizing, it will be not benifitial.

- I tried to see how cards would shuffle and in 7 levels what would be best on average, because at first our hand is not that high, I decided to begin with d, s is for when we already have what we wanted, if this was not the case, I would at first check for deleting opponents card and the self deleting, at first this was totally opposit but I used them in this sequence and it got better. 




More info:


Idea: we can often compute the utility of the root without examining all the nodes in the game tree.
Once we've seen some of the lefthand children of a node, this gives us a preliminary value for that node. For a max node, this is a lower bound on the value at the node. For a min node, this gives us an upper bound. Therefore, as we start to look at additional children, we can abandon the exploration once it becomes clear that the new child can't beat the value that we're currently holding.

Look through this pruning example from Diane Litman (U. Pitt).

As you can see from this example, this strategy can greatly reduce the amount of the tree we have to explore. Remember that we're building the game tree dynamically as we do depth-first search, so "don't have to explore" means that we never even allocate those nodes of the tree.

Notice that the left-to-right order of nodes matters to the performance of alpha-beta pruning. See this example (from Mitch Marcus (U. Penn), adapted from Rick Lathrop (USC)). Remember that the tree is built dynamically, so this left-to-right ordering is determined by the order in which we consider the various possible actions (moves in the game).

Suppose that we have b moves in each game state (i.e. branching factor b), and our tree has height m. Then standard minimax examines O(bm) nodes. If we visit the nodes in the optimal order, alpha-beta pruning will reduce the number of nodes searched to O(bm/2). If we visit the nodes in random order, we'll examine O(b3m/4) on average.

Obviously, we can't magically arrange to have the nodes in exactly the right order. However, heuristic ordering of actions can get close to the optimal O(b^{m/2}) in practice. For example, a good heuristic order for chess examines possible moves in the following order:

(first) captures
threats
forward moves
(last) backwards moves

Resource: https://courses.engr.illinois.edu/cs440/fa2018/lectures/lect33.html